# Load Libraries

In [1]:
# Load libraries
library(readxl)
library(dplyr)
library(readr)
# Inflation rate: https://fred.stlouisfed.org/series/FPCPITOTLZGUSA


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Part 1: Clean Population Data

In [2]:
pop_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/population.csv")
glimpse(pop_raw)

Rows: 264
Columns: 2
$ DATE   <fct> 1959-01-01, 1959-04-01, 1959-07-01, 1959-10-01, 1960-01-01, 196…
$ POPTHM <fct> 176045.333333333333, 176726.666666666667, 177480.666666666667, …


In [3]:
sum(is.na(pop_raw$POPTHM))

[1] 0

In [4]:
# Creating new column population
pop_raw$population <- pop_raw$POPTHM

# Making column pop into a numeric
pop_raw$population <- as.numeric(as.character(pop_raw$POPTHM))

# Converting pop into the right unit by multiplything by 1000
pop_raw$population <- pop_raw$population * 1000

Warning message:
“NAs introduced by coercion”


In [5]:
# Time as a date
pop_raw$DATE <- as.character(pop_raw$DATE)
pop_raw$time_as_date <- as.Date(pop_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
pop_raw$time_as_numeric <- as.numeric(pop_raw$time_as_date)

In [6]:
# Filter data between 1984-01-01 and 2024-04-01
pop_raw <- pop_raw[pop_raw$time_as_date >= as.Date("1984-01-01") & pop_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "population")
pop_filtered <- pop_raw[, columns_to_keep]

# Check
glimpse(pop_filtered)
sum(is.na(pop_filtered))

Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ population      <dbl> 235604667, 236082333, 236657333, 237232667, 237672667,…


[1] 0

# Part 2: Clean Unemployment Rate

In [7]:
unemployment_rate_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/unemployement_rate.csv")
glimpse(unemployment_rate_raw)

Rows: 308
Columns: 2
$ DATE   <fct> 1948-01-01, 1948-04-01, 1948-07-01, 1948-10-01, 1949-01-01, 194…
$ UNRATE <fct> 3.7333333333333333, 3.6666666666666667, 3.7666666666666667, 3.8…


In [8]:
# Creating new column for rate
unemployment_rate_raw$unemployment_rate <- unemployment_rate_raw$UNRATE

# Making column pop into a numeric
unemployment_rate_raw$unemployment_rate <- as.numeric(as.character(unemployment_rate_raw$UNRATE))

# Converting pop into a decima since it's in percentage 
unemployment_rate_raw$unemployment_rate <- unemployment_rate_raw$unemployment_rate / 100

# Time as a date
unemployment_rate_raw$DATE <- as.character(unemployment_rate_raw$DATE)
unemployment_rate_raw$time_as_date <- as.Date(unemployment_rate_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
unemployment_rate_raw$time_as_numeric <- as.numeric(unemployment_rate_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
unemployment_rate_raw <- unemployment_rate_raw[unemployment_rate_raw$time_as_date >= as.Date("1984-01-01") & unemployment_rate_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "unemployment_rate")
unemployment_rate_filtered <- unemployment_rate_raw[, columns_to_keep]

# Check
glimpse(unemployment_rate_filtered)
sum(is.na(unemployment_rate_filtered))


Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date      <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 198…
$ time_as_numeric   <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844…
$ unemployment_rate <dbl> 0.07866667, 0.07433333, 0.07433333, 0.07300000, 0.07…


[1] 0

# Part 3: Clean Consumer Price Index

In [9]:
# Source: https://fred.stlouisfed.org/series/CPALTT01USM657N#0
CPI_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/consumer_price_index.csv")
glimpse(CPI_raw)

Rows: 276
Columns: 2
$ DATE            <fct> 1955-04-01, 1955-07-01, 1955-10-01, 1956-01-01, 1956-0…
$ CPALTT01USQ657N <dbl> 0.0000000, 0.4993758, 0.1242236, -0.2481390, 0.8706468…


In [10]:
# Convert ConsumerPriceIndex into a numeric column
CPI_raw$ConsumerPriceIndex <- as.numeric(as.character(CPI_raw$CPALTT01USQ657N))

# Convert DATE column to a Date format
CPI_raw$DATE <- as.character(CPI_raw$DATE)
CPI_raw$time_as_date <- as.Date(CPI_raw$DATE, format = "%Y-%m-%d")

# Convert time_as_date to numeric for modeling purposes
CPI_raw$time_as_numeric <- as.numeric(CPI_raw$time_as_date)

# Filter data to include only rows between 1984-01-01 and 2024-04-01
CPI_raw <- CPI_raw[CPI_raw$time_as_date >= as.Date("1984-01-01") & 
                     CPI_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "ConsumerPriceIndex")
CPI_filtered <- CPI_raw[, columns_to_keep]


# Check the updated dataset
glimpse(CPI_filtered)
sum(is.na(CPI_filtered))

Rows: 161
Columns: 3
$ time_as_date       <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 19…
$ time_as_numeric    <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 584…
$ ConsumerPriceIndex <dbl> 1.1202636, 1.0752688, 1.0960671, 0.7334184, 0.63311…


[1] 0

In [11]:
str(CPI_filtered)
tail(CPI_filtered)

'data.frame':	161 obs. of  3 variables:
 $ time_as_date      : Date, format: "1984-01-01" "1984-04-01" ...
 $ time_as_numeric   : num  5113 5204 5295 5387 5479 ...
 $ ConsumerPriceIndex: num  1.12 1.075 1.096 0.733 0.633 ...


,time_as_date,time_as_numeric,ConsumerPriceIndex
,<date>,<dbl>,<dbl>
271,2022-10-01,19266,0.3671613
272,2023-01-01,19358,1.0449066
273,2023-04-01,19448,1.1923322
274,2023-07-01,19539,0.8664265
275,2023-10-01,19631,0.1045077
276,2024-01-01,19723,1.0425755


# Part 4: Disposable Personal Income

In [12]:
disposable_income_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/disposable_personal_income.csv")
glimpse(disposable_income_raw)
# https://fred.stlouisfed.org/series/DSPI

Rows: 260
Columns: 2
$ DATE     <fct> 1960-01-01, 1960-04-01, 1960-07-01, 1960-10-01, 1961-01-01, 1…
$ DSPI_PC1 <fct> 5.13401, 4.26418, 4.34502, 3.47304, 3.31239, 3.64620, 4.81694…


In [13]:
# Creating new column for disposable personal income
disposable_income_raw$DisposablePersonalIncome <- disposable_income_raw$DSPI_PC1

# Making column disposable personal income numeric
disposable_income_raw$DisposablePersonalIncome <- as.numeric(as.character(disposable_income_raw$DSPI_PC1))

# Converting dis income into true unites it is in trillions so multiply by billions
disposable_income_raw$DisposablePersonalIncome <- disposable_income_raw$DisposablePersonalIncome * 1000000000

# Time as a date
disposable_income_raw$DATE <- as.character(disposable_income_raw$DATE)
disposable_income_raw$time_as_date <- as.Date(disposable_income_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
disposable_income_raw$time_as_numeric <- as.numeric(disposable_income_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
disposable_income_raw <- disposable_income_raw[disposable_income_raw$time_as_date >= as.Date("1984-01-01") & disposable_income_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "DisposablePersonalIncome")
disposable_income_filtered <- disposable_income_raw[, columns_to_keep]

# Check
glimpse(disposable_income_filtered)
sum(is.na(disposable_income_filtered))


Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date             <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-…
$ time_as_numeric          <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 575…
$ DisposablePersonalIncome <dbl> 10729270000, 11779660000, 10982020000, 947008…


[1] 0

# Part 5: Cleaing 10 Year Treasury Yield

In [14]:
treasury_yield_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/10_year_treasury_yield.csv")
glimpse(treasury_yield_raw)

Rows: 286
Columns: 2
$ DATE            <fct> 1953-04-01, 1953-07-01, 1953-10-01, 1954-01-01, 1954-0…
$ IRLTLT01USQ156N <dbl> 2.996667, 2.916667, 2.643333, 2.440000, 2.346667, 2.34…


In [15]:
# Creating new column for yield
treasury_yield_raw$Yield <- treasury_yield_raw$IRLTLT01USQ156N

# Making column yield numeric
treasury_yield_raw$Yield <- as.numeric(as.character(treasury_yield_raw$IRLTLT01USQ156N))

# Converting dis income into true unites by making it a decimal
treasury_yield_raw$Yield <- treasury_yield_raw$Yield / 100

# Time as a date
treasury_yield_raw$DATE <- as.character(treasury_yield_raw$DATE)
treasury_yield_raw$time_as_date <- as.Date(treasury_yield_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
treasury_yield_raw$time_as_numeric <- as.numeric(treasury_yield_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
treasury_yield_raw <- treasury_yield_raw[treasury_yield_raw$time_as_date >= as.Date("1984-01-01") & treasury_yield_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "Yield")
treasury_yield_filtered <- treasury_yield_raw[, columns_to_keep]

# Check
glimpse(treasury_yield_filtered)
sum(is.na(treasury_yield_filtered))


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ Yield           <dbl> 0.11943333, 0.13200000, 0.12866667, 0.11743333, 0.1158…


[1] 0

# Part 6: Clean Fed Funds Rate Data

In [16]:
fed_rate_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/fed_funds_rate.csv")
glimpse(fed_rate_raw)
# https://fred.stlouisfed.org/series/FEDFUNDS

Rows: 282
Columns: 2
$ DATE     <fct> 1954-07-01, 1954-10-01, 1955-01-01, 1955-04-01, 1955-07-01, 1…
$ FEDFUNDS <fct> 1.03000000000000000000, 0.98666666666666666667, 1.34333333333…


In [17]:
# Creating new column for yield
fed_rate_raw$fed_rate <- fed_rate_raw$FEDFUNDS

# Making column yield numeric
fed_rate_raw$fed_rate <- as.numeric(as.character(fed_rate_raw$FEDFUNDS))

# Converting rate into true unites by making it a decimal
fed_rate_raw$fed_rate <- fed_rate_raw$fed_rate / 100

# Time as a date
fed_rate_raw$DATE <- as.character(fed_rate_raw$DATE)
fed_rate_raw$time_as_date <- as.Date(fed_rate_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
fed_rate_raw$time_as_numeric <- as.numeric(fed_rate_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
fed_rate_raw <- fed_rate_raw[fed_rate_raw$time_as_date >= as.Date("1984-01-01") & fed_rate_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "fed_rate")
fed_rate_filtered <- fed_rate_raw[, columns_to_keep]

# Check
glimpse(fed_rate_filtered)
sum(is.na(fed_rate_filtered))


Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ fed_rate        <dbl> 0.09686667, 0.10556667, 0.11390000, 0.09266667, 0.0847…


[1] 0

# Part 7: Clean GDP Data

In [18]:
# https://fred.stlouisfed.org/series/GDP
GDP_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/GDP.csv")
glimpse(GDP_raw)


Rows: 311
Columns: 2
$ DATE <fct> 1947-01-01, 1947-04-01, 1947-07-01, 1947-10-01, 1948-01-01, 1948-…
$ GDP  <dbl> 243.164, 245.968, 249.585, 259.745, 265.742, 272.567, 279.196, 28…


In [19]:

# Making column GDP numeric
GDP_raw$GDP <- as.numeric(as.character(GDP_raw$GDP))

# Converting dis income into true unites by making it a decimal
GDP_raw$GDP <- GDP_raw$GDP * 1000000000

# Time as a date
GDP_raw$DATE <- as.character(GDP_raw$DATE)
GDP_raw$time_as_date <- as.Date(GDP_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
GDP_raw$time_as_numeric <- as.numeric(GDP_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
GDP_raw <- GDP_raw[GDP_raw$time_as_date >= as.Date("1984-01-01") & GDP_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "GDP")
GDP_filtered <- GDP_raw[, columns_to_keep]

# Check
glimpse(GDP_filtered)
sum(is.na(GDP_filtered))


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ GDP             <dbl> 3.908054e+12, 4.009601e+12, 4.084250e+12, 4.148551e+12…


[1] 0

In [20]:
# Add GDP growth rate column
GDP_filtered <- GDP_filtered %>%
  mutate(
    GDP_growth_rate = (GDP - lag(GDP)) / lag(GDP)
  )

# View the result
glimpse(GDP_filtered)
sum(is.na(GDP_filtered))



Rows: 162
Columns: 4
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ GDP             <dbl> 3.908054e+12, 4.009601e+12, 4.084250e+12, 4.148551e+12…
$ GDP_growth_rate <dbl> NA, 0.025984032, 0.018617563, 0.015743649, 0.019673616…


[1] 1

# Part 8: Clean Loan Data

In [21]:
# https://fred.stlouisfed.org/series/TOTCI
loan_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/Loans.csv")
glimpse(loan_raw)


Rows: 208
Columns: 2
$ DATE  <fct> 1973-01-01, 1973-04-01, 1973-07-01, 1973-10-01, 1974-01-01, 1974…
$ TOTCI <fct> 139.9047000000000000, 149.8688000000000000, 157.0841538461538462…


In [22]:

# Making column loan numeric
loan_raw$loan <- as.numeric(as.character(loan_raw$TOTCI))

# Converting this loan into billions 
loan_raw$loan <- loan_raw$loan * 1000000000

# Time as a date
loan_raw$DATE <- as.character(loan_raw$DATE)
loan_raw$time_as_date <- as.Date(loan_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
loan_raw$time_as_numeric <- as.numeric(loan_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
loan_raw <- loan_raw[loan_raw$time_as_date >= as.Date("1984-01-01") & loan_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "loan")
loan_filtered <- loan_raw[, columns_to_keep]

# Check
glimpse(loan_filtered)
sum(is.na(loan_filtered))


Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ loan            <dbl> 422715769231, 444453800000, 458430284615, 469857400000…


[1] 0

# Part 9: Clean Money Supply Data

In [23]:
# Source: https://fred.stlouisfed.org/series/MABMM301USM189S#0
money_supply_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/money_supply.csv")
glimpse(money_supply_raw)

Rows: 256
Columns: 2
$ DATE            <fct> 1960-01-01, 1960-04-01, 1960-07-01, 1960-10-01, 1961-0…
$ MABMM301USM189S <fct> 298633333333.33333333, 301100000000.00000000, 30646666…


In [24]:
# Making column money supply numeric
money_supply_raw$money_supply <- as.numeric(as.character(money_supply_raw$MABMM301USM189S))

# Time as a date
money_supply_raw$DATE <- as.character(money_supply_raw$DATE)
money_supply_raw$time_as_date <- as.Date(money_supply_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
money_supply_raw$time_as_numeric <- as.numeric(money_supply_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
money_supply_raw <- money_supply_raw[money_supply_raw$time_as_date >= as.Date("1984-01-01") & money_supply_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "money_supply")
money_supply_filtered <- money_supply_raw[, columns_to_keep]

# Check
glimpse(money_supply_filtered)
sum(is.na(money_supply_filtered))
money_supply_filtered

Warning message:
“NAs introduced by coercion”


Rows: 160
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ money_supply    <dbl> 2.157200e+12, 2.203633e+12, 2.232767e+12, 2.282233e+12…


[1] 1

,time_as_date,time_as_numeric,money_supply
,<date>,<dbl>,<dbl>
97,1984-01-01,5113,2.157200e+12
98,1984-04-01,5204,2.203633e+12
99,1984-07-01,5295,2.232767e+12
100,1984-10-01,5387,2.282233e+12
101,1985-01-01,5479,2.350900e+12
102,1985-04-01,5569,2.392500e+12
103,1985-07-01,5660,2.443300e+12
104,1985-10-01,5752,2.479300e+12
105,1986-01-01,5844,2.516033e+12


# Part 10: Clean Reserves of Depository

In [25]:
# Source: https://fred.stlouisfed.org/series/TOTRESNS#0
depository_reserve_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/reserves_deposit.csv")
glimpse(depository_reserve_raw)

Rows: 264
Columns: 2
$ DATE     <fct> 1959-01-01, 1959-04-01, 1959-07-01, 1959-10-01, 1960-01-01, 1…
$ TOTRESNS <fct> 18.6333333333333333, 18.6000000000000000, 18.6333333333333333…


In [26]:
# Making column loan numeric
depository_reserve_raw$deposit_reserve<- as.numeric(as.character(depository_reserve_raw$TOTRESNS))

# Converting this loan into billions 
depository_reserve_raw$deposit_reserve <- depository_reserve_raw$deposit_reserve * 1000000000

# Time as a date
depository_reserve_raw$DATE <- as.character(depository_reserve_raw$DATE)
depository_reserve_raw$time_as_date <- as.Date(depository_reserve_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
depository_reserve_raw$time_as_numeric <- as.numeric(depository_reserve_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
depository_reserve_raw <- depository_reserve_raw[depository_reserve_raw$time_as_date >= as.Date("1984-01-01") & depository_reserve_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "deposit_reserve")
depository_reserve_filtered <- depository_reserve_raw[, columns_to_keep]

# Check
glimpse(depository_reserve_filtered)
sum(is.na(depository_reserve_filtered))

Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ deposit_reserve <dbl> 37533333333, 37033333333, 37600000000, 39466666667, 40…


[1] 0

# Part 11: Consumer Sentimnent Index

In [27]:
# Source: https://fred.stlouisfed.org/series/UMCSENT#0
consumer_sentiment_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/consumer_sentiment_index.csv")
glimpse(consumer_sentiment_raw)

Rows: 288
Columns: 2
$ DATE    <fct> 1953-01-01, 1953-04-01, 1953-07-01, 1953-10-01, 1954-01-01, 19…
$ UMCSENT <fct> 90.7000000000000000, ., 80.8000000000000000, 80.70000000000000…


In [28]:
# Making column loan numeric
consumer_sentiment_raw$consumer_sentiment<- as.numeric(as.character(consumer_sentiment_raw$UMCSENT))

# Time as a date
consumer_sentiment_raw$DATE <- as.character(consumer_sentiment_raw$DATE)
consumer_sentiment_raw$time_as_date <- as.Date(consumer_sentiment_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
consumer_sentiment_raw$time_as_numeric <- as.numeric(consumer_sentiment_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
consumer_sentiment_raw <- consumer_sentiment_raw[consumer_sentiment_raw$time_as_date >= as.Date("1984-01-01") & consumer_sentiment_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "consumer_sentiment")
consumer_sentiment_filtered <- consumer_sentiment_raw[, columns_to_keep]

# Check
glimpse(consumer_sentiment_filtered)
sum(is.na(consumer_sentiment_filtered))

Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date       <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 19…
$ time_as_numeric    <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 584…
$ consumer_sentiment <dbl> 99.50000, 96.56667, 98.86667, 94.96667, 94.46667, 9…


[1] 0

# Part 12: Clean Homeownership Rate

In [29]:
# Source: https://fred.stlouisfed.org/series/RHORUSQ156N#0
homeownership_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/homeownership_rate.csv")
glimpse(homeownership_raw)

Rows: 239
Columns: 2
$ DATE        <fct> 1965-01-01, 1965-04-01, 1965-07-01, 1965-10-01, 1966-01-01…
$ RHORUSQ156N <dbl> 62.9, 62.9, 62.9, 63.4, 63.5, 63.2, 63.3, 63.8, 63.3, 63.9…


In [30]:
# Making column loan numeric
homeownership_raw$homeownership_rate<- as.numeric(as.character(homeownership_raw$RHORUSQ156N))

# Converting rate into true unites by making it a decimal
homeownership_raw$homeownership_rate <- homeownership_raw$homeownership_rate / 100

# Time as a date
homeownership_raw$DATE <- as.character(homeownership_raw$DATE)
homeownership_raw$time_as_date <- as.Date(homeownership_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
homeownership_raw$time_as_numeric <- as.numeric(homeownership_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
homeownership_raw <- homeownership_raw[homeownership_raw$time_as_date >= as.Date("1984-01-01") & homeownership_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "homeownership_rate")
homeownership_filtered <- homeownership_raw[, columns_to_keep]

# Check
glimpse(homeownership_filtered)
sum(is.na(homeownership_filtered))

Rows: 162
Columns: 3
$ time_as_date       <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 19…
$ time_as_numeric    <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 584…
$ homeownership_rate <dbl> 0.646, 0.646, 0.646, 0.641, 0.641, 0.641, 0.639, 0.…


[1] 0

# Part 13: Clean CBOE Volatility Index VIX

In [31]:
vix_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/VIX.csv")
glimpse(vix_raw)

Rows: 140
Columns: 2
$ DATE   <fct> 1990-01-01, 1990-04-01, 1990-07-01, 1990-10-01, 1991-01-01, 199…
$ VIXCLS <fct> 22.1746031746031746, 18.7220634920634921, 25.1955555555555556, …


In [32]:
# Making column loan numeric
vix_raw$vix_index<- as.numeric(as.character(vix_raw$VIXCLS))

# Time as a date
vix_raw$DATE <- as.character(vix_raw$DATE)
vix_raw$time_as_date <- as.Date(vix_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
vix_raw$time_as_numeric <- as.numeric(vix_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
vix_raw <- vix_raw[vix_raw$time_as_date >= as.Date("1984-01-01") & vix_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "vix_index")
vix_filtered <- vix_raw[, columns_to_keep]

# Check
glimpse(vix_filtered)
sum(is.na(vix_filtered))

Warning message:
“NAs introduced by coercion”


Rows: 138
Columns: 3
$ time_as_date    <date> 1990-01-01, 1990-04-01, 1990-07-01, 1990-10-01, 1991-…
$ time_as_numeric <dbl> 7305, 7395, 7486, 7578, 7670, 7760, 7851, 7943, 8035, …
$ vix_index       <dbl> 22.17460, 18.72206, 25.19556, 26.11328, 22.51583, 17.1…


[1] 0

# Part 14: NASDAQ Composite Index

In [33]:
# Source: https://fred.stlouisfed.org/series/NASDAQCOM#0
NASDAQ_raw <- read.csv("/workspaces/DUKE_MIDS_QFC_Final_Project/dataset/NASDAQ.csv")
glimpse(NASDAQ_raw)

Rows: 215
Columns: 2
$ DATE      <fct> 1971-04-01, 1971-07-01, 1971-10-01, 1972-01-01, 1972-04-01, …
$ NASDAQCOM <fct> 109.2152380952380952, 108.4165625000000000, 107.307500000000…


In [34]:
# Making column index numeric
NASDAQ_raw$NASDAQ_index<- as.numeric(as.character(NASDAQ_raw$NASDAQCOM))

# Time as a date
NASDAQ_raw$DATE <- as.character(NASDAQ_raw$DATE)
NASDAQ_raw$time_as_date <- as.Date(NASDAQ_raw$DATE, format = "%Y-%m-%d")

# Time as numeric (used in certain regression models or methods)
NASDAQ_raw$time_as_numeric <- as.numeric(NASDAQ_raw$time_as_date)

# Filter data between 1984-01-01 and 2024-04-01
NASDAQ_raw <- NASDAQ_raw[NASDAQ_raw$time_as_date >= as.Date("1984-01-01") & NASDAQ_raw$time_as_date <= as.Date("2024-04-01"), ]

# Keep only the relevant columns
columns_to_keep <- c("time_as_date", "time_as_numeric", "NASDAQ_index")
NASDAQ_filtered <- NASDAQ_raw[, columns_to_keep]

# Check
glimpse(NASDAQ_filtered)
sum(is.na(NASDAQ_filtered))

Warning message:
“NAs introduced by coercion”


Rows: 162
Columns: 3
$ time_as_date    <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-01, 1985-…
$ time_as_numeric <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 5752, 5844, …
$ NASDAQ_index    <dbl> 262.4971, 242.9894, 244.8656, 245.2678, 275.1484, 286.…


[1] 0

# Part 15: Group all data into one dataframe

In [35]:
# Merge two datasets by the 'Date' column
datasets_to_merge <- list(pop_filtered, unemployment_rate_filtered, CPI_filtered, disposable_income_filtered, treasury_yield_filtered, fed_rate_filtered, GDP_filtered)

# Re-run the merge
regressors_data <- Reduce(function(x, y) {
  merge(x, y, by = c("time_as_date", "time_as_numeric"), all = TRUE)
}, datasets_to_merge)

glimpse(regressors_data)
regressors_data



Rows: 162
Columns: 10
$ time_as_date             <date> 1984-01-01, 1984-04-01, 1984-07-01, 1984-10-…
$ time_as_numeric          <dbl> 5113, 5204, 5295, 5387, 5479, 5569, 5660, 575…
$ population               <dbl> 235604667, 236082333, 236657333, 237232667, 2…
$ unemployment_rate        <dbl> 0.07866667, 0.07433333, 0.07433333, 0.0730000…
$ ConsumerPriceIndex       <dbl> 1.1202636, 1.0752688, 1.0960671, 0.7334184, 0…
$ DisposablePersonalIncome <dbl> 10729270000, 11779660000, 10982020000, 947008…
$ Yield                    <dbl> 0.11943333, 0.13200000, 0.12866667, 0.1174333…
$ fed_rate                 <dbl> 0.09686667, 0.10556667, 0.11390000, 0.0926666…
$ GDP                      <dbl> 3.908054e+12, 4.009601e+12, 4.084250e+12, 4.1…
$ GDP_growth_rate          <dbl> NA, 0.025984032, 0.018617563, 0.015743649, 0.…


time_as_date,time_as_numeric,population,unemployment_rate,ConsumerPriceIndex,DisposablePersonalIncome,Yield,fed_rate,GDP,GDP_growth_rate
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1984-01-01,5113,235604667,0.07866667,1.1202636,10729270000,0.11943333,0.09686667,3.908054e+12,NA
1984-04-01,5204,236082333,0.07433333,1.0752688,11779660000,0.13200000,0.10556667,4.009601e+12,0.025984032
1984-07-01,5295,236657333,0.07433333,1.0960671,10982020000,0.12866667,0.11390000,4.084250e+12,0.018617563
1984-10-01,5387,237232667,0.07300000,0.7334184,9470080000,0.11743333,0.09266667,4.148551e+12,0.015743649
1985-01-01,5479,237672667,0.07233333,0.6331117,7253910000,0.11583333,0.08476667,4.230168e+12,0.019673616
1985-04-01,5569,238175667,0.07300000,1.2268009,7486790000,0.10813333,0.07923333,4.294887e+12,0.015299392
1985-07-01,5660,238789333,0.07200000,0.7147296,5819840000,0.10336667,0.07900000,4.386773e+12,0.021394276
1985-10-01,5752,239386667,0.07033333,0.8947856,6193650000,0.09760000,0.08103333,4.444094e+12,0.013066781
1986-01-01,5844,239860667,0.07033333,0.2140673,7391500000,0.08556667,0.07826667,4.507894e+12,0.014356132


In [36]:
saveRDS(regressors_data, "regressors_data.rds")  # Save the data to call it in other notebooks